# Process Data from files

- Pre requisities: have multiple files with loops

# Date time helper functions

In [2]:
# Turn Avisi Datetime string to python DateTime object
def avisi_str_to_datetime(avisi_str):
    dt_pattern = pattern = '%Y-%m-%dT%H:%M:%S.%fZ' # pattern of how api passes date times. The Z at the end indicates UTC time
    return dt.datetime.strptime(avisi_str, dt_pattern)

# express date time as a floating number
def datetime_to_float(d):
    return d.timestamp()

def float_to_datetime(f):
    return dt.datetime.fromtimestamp(f)

def datetime_to_avisi_str(dtime):
    return dtime.strftime('%Y-%m-%dT%H:%M:%S.%fZ')

# Plotting data from file in line

In [5]:
def get_data_x_position_coordinates_by_file_name(file_name):
    big_data_x_coordinates = []

    with open(file_name, 'r') as file:
        lines = file.readlines()

        for line in lines:
            coordinate = []
            json_line = json.loads(line)

            datetime_obj = avisi_str_to_datetime(json_line['timestamp'])
            datetime_as_float = datetime_to_float(datetime_obj)

            coordinate.append(datetime_as_float)
            coordinate.append(json_line['x_position'])
            big_data_x_coordinates.append(coordinate)

    return big_data_x_coordinates

def get_data_y_position_coordinates_by_file_name(file_name):
    big_data_y_coordinates = []

    with open(file_name, 'r') as file:
        lines = file.readlines()

        for line in lines:
            coordinate = []
            json_line = json.loads(line)

            datetime_obj = avisi_str_to_datetime(json_line['timestamp'])
            datetime_as_float = datetime_to_float(datetime_obj)

            coordinate.append(datetime_as_float)
            coordinate.append(json_line['y_position'])
            big_data_y_coordinates.append(coordinate)

    return big_data_y_coordinates

def plot_rat_run_by_file(file_name, x_or_y = "x", plot_name="NO_NAME"):
    full_coords = []
    
    if x_or_y == "x":
        full_coords = get_data_x_position_coordinates_by_file_name(file_name)
    else:
        full_coords = get_data_y_position_coordinates_by_file_name(file_name)

    plt.plot(*zip(*full_coords))
    plt.title(plot_name)

In [ ]:
plot_rat_run_by_file(RAT_RUN_NAME, "x", "NAME")

# Actual plotting

In [6]:
total_duration_as_float_all = []

# The input of this function should be exactly 1 completed rat run in a txt file
def print_single_rat_run_info(file_location):
    with open(file_location, 'r') as file:
        lines = file.readlines()

        first_line = json.loads(lines[0])
        last_line = json.loads(lines[len(lines)-1])
        
        first_line_timestamp_float = datetime_to_float(avisi_str_to_datetime(first_line['timestamp']))
        last_line_timestamp_float = datetime_to_float(avisi_str_to_datetime(last_line['timestamp']))
        
        total_duration_as_float = last_line_timestamp_float - first_line_timestamp_float
        total_duration_as_float_all.append(total_duration_as_float)
        
        start_dt_object = dt.datetime.fromtimestamp(first_line_timestamp_float)
        end_dt_object = dt.datetime.fromtimestamp(last_line_timestamp_float)
        
        print(file_location + " duration: " + str(end_dt_object - start_dt_object) + " as float: " + str(total_duration_as_float))

def print_all_single_rat_runs_in_folder(folder_location, n_of_runs):
    file_location_and_name = folder_location + "/complete_rat_run" + str(index) + ".txt"
    
    for x in range(0, n_of_runs):
        print("Doing rat_run " + str(x))
        print_single_rat_run_info(file_location_and_name)

In [ ]:
# Call
print_all_single_rat_runs_in_folder(FOLDER_NAME, N_RUNS)